# Lab 3: Build a Search Agent

In this lab, we'll use the Azure AI Agent Service to create an agent that is able to retrieve information from documents stored in Azure AI Search, a vector database. This pattern is known as retrieval augmented generation or RAG. The documents that we'll be searching are health insurance policies.

#### Step 1: Load packages

In [ ]:
import os
from dotenv import load_dotenv
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
from azure.ai.agents.models import AzureAISearchToolDefinition, AzureAISearchToolResource, AISearchIndexResource, ToolResources, AgentThreadCreationOptions

load_dotenv()

#### Step 2: Connect to your Azure AI Foundry project

In [ ]:
# Connecting to our Azure AI Foundry project, which will allow us to use the deployed gpt-4o model
project = AIProjectClient(
    endpoint=os.getenv("AIPROJECT_ENDPOINT"),
    credential=DefaultAzureCredential()
)

#### Step 3: Connect to your Azure AI Search index

In [ ]:
# First enter the name of your search index

index_name="health-plan"
print(index_name)

This code retrieves the connection ID for your Azure AI Search resource, then defines and configures the search tool and its resources. It ensures your agent is set up to access the correct search index, enabling it to perform document retrieval using the Azure AI Agent Service SDK.

In [ ]:
# Iterate through the connections in your project and get the connection ID of the Azure AI Search connection.
conn_id = None
for conn in project.connections.list():
    if getattr(conn, "type", None) == "CognitiveSearch":
        conn_id = conn.id
        break

if not conn_id:
    raise ValueError("No Azure Cognitive Search connection found in this project.")

# Define the Azure AI Search tool and its resources using the latest SDK classes, and connect to your AI Search index
ai_search_tool = AzureAISearchToolDefinition()
ai_search_resource = AzureAISearchToolResource(
    index_list=[
        AISearchIndexResource(
            index_connection_id=conn_id,
            index_name=index_name # Be sure to set your index name above
        )
    ]
)

# The tool resources are used to define the tools available to the agent
tool_resources = ToolResources(azure_ai_search=ai_search_resource)

#### Step 4: Define the search agent

In this step, you will define and create the search agent using the Azure AI Agent Service SDK. The agent is configured with the GPT-4o model, a descriptive name, instructions for its behavior, and the search tool and resources you set up previously. This setup enables the agent to process user queries and retrieve relevant information from your Azure AI Search index, making it capable of intelligent, document-grounded search.

In [ ]:
search_agent = project.agents.create_agent(
    model="gpt-4o",
    name="search-agent",
    instructions="You are a helpful agent that is an expert at searching health plan documents.",
    tools=[ai_search_tool],
    tool_resources=tool_resources
)

#### Step 5: Chat with the search agent

In this step, you'll interact with your search agent by sending it a user query and processing its response. The code demonstrates how to:
- Create a conversation thread with an initial user message
- Run the agent to process the query using the Azure AI Agent Service SDK
- Retrieve and display the agent's response, which is grounded in the indexed health plan documents
- Clean up by deleting the agent after use

This hands-on interaction shows how retrieval-augmented generation (RAG) enables your agent to provide accurate, document-based answers to natural language questions.

In [ ]:
# The name of the health plan we want to search for
plan_name = 'Northwind Standard'

# Create thread options with initial user message and tool resources (use dict for message)
thread_options = AgentThreadCreationOptions(
    messages=[
        {
            "role": "user",
            "content": f"Tell me about the {plan_name} plan."
        }
    ],
    tool_resources=tool_resources
)

# Use create_thread_and_process_run to create the thread, message, and run in one step
run = project.agents.create_thread_and_process_run(
    agent_id=search_agent.id,
    thread=thread_options
)

# Check if the run was successful
if run.status == "failed":
    print(f"Run failed: {run.last_error}")

# Delete the agent when it's done running
project.agents.delete_agent(search_agent.id)

# Fetch all the messages from the thread
messages = project.agents.messages.list(thread_id=run.thread_id)

# Print the last assistant/agent message's text, if any
last_msg = None
for msg in reversed(list(messages)):
    role = getattr(msg, "role", None)
    if role and ("agent" in role.lower() or "assistant" in role.lower()):
        last_msg = msg
        break

if last_msg and getattr(last_msg, "content", None) and isinstance(last_msg.content, list):
    for part in last_msg.content:
        if part.get("type") == "text" and "text" in part and "value" in part["text"]:
            print('Agent:', part["text"]["value"])
            break